<a href="https://colab.research.google.com/github/lgachig/recuperaci-n/blob/main/Recuperaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import Counter
import re

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_Latin_phrases_(full)"

In [6]:
reponse = requests.get(url)
soup = BeautifulSoup(reponse.text, "html.parser")
tablas = soup.find_all('table')

In [7]:
todas_las_filas = []

for tabla in tablas:
    filas = []
    for tr in tabla.find_all('tr'):
        celdas = []
        for th in tr.find_all('th'):
            colspan = int(th.get('colspan', 1))
            texto = th.get_text(strip=True)
            celdas.extend([texto] * colspan)
        for td in tr.find_all('td'):
            colspan = int(td.get('colspan', 1))
            texto = td.get_text(strip=True)
            celdas.extend([texto] * colspan)
        if celdas:
            filas.append(celdas)
    if len(filas) > 1:
        encabezado = filas[0]
        for fila in filas[1:]:
            if len(fila) == len(encabezado):
                todas_las_filas.append(dict(zip(encabezado, fila)))

df= pd.DataFrame(todas_las_filas)

In [12]:
df.to_excel("latin_phrases.xlsx", index=False)

In [1]:
!pip install cltk

In [2]:
import pandas as pd
import spacy
from collections import Counter
import re
from cltk.languages.example_texts import get_example_text
from cltk.core.data_types import Pipeline
from cltk import NLP

In [14]:
nlp_en = spacy.load("en_core_web_sm")

# Cargar CLTK para latín
nlp_la = NLP(language="lat", suppress_banner=True)

# --- Funciones ---

# Contar palabras y verbos en inglés
def contar_palabras_y_verbos_ingles(textos):
    palabras = []
    verbos = []
    for doc in nlp_en.pipe(textos, disable=["ner"]):
        for token in doc:
            if token.is_alpha and not token.is_stop:
                palabras.append(token.lemma_.lower())
                if token.pos_ == "VERB":
                    verbos.append(token.lemma_.lower())
    return Counter(palabras), Counter(verbos)

def contar_palabras_y_verbos_latin(textos):
    palabras = []
    verbos = []
    for texto in textos:
        doc = nlp_la.analyze(texto)
        for word in doc.words:
            # Obtener pos y lemma de forma segura
            pos_str = str(word.pos).lower() if word.pos else ""
            lemma_str = word.lemma.lower() if word.lemma else ""

            # Filtrar solo palabras (sin símbolos, sin números)
            if lemma_str.isalpha() and len(lemma_str) > 2:
                if 'verb' in pos_str:
                    verbos.append(lemma_str)
                else:
                    palabras.append(lemma_str)

    return Counter(palabras), Counter(verbos)

# --- Cargar datos ---
latin = df["Latin"].dropna().tolist()
english = df["Translation"].dropna().tolist()

# --- Procesar textos ---
palabras_latin, verbos_latin = contar_palabras_y_verbos_latin(latin)
palabras_english, verbos_english = contar_palabras_y_verbos_ingles(english)

# --- Mostrar resultados ---
print("🔵 Top 10 palabras en LATÍN:")
print(palabras_latin.most_common(10))

print("\n🟣 Top 10 VERBOS en LATÍN:")
print(verbos_latin.most_common(10))

print("\n🟢 Top 10 palabras en INGLÉS:")
print(palabras_english.most_common(10))

print("\n🟠 Top 10 VERBOS en INGLÉS:")
print(verbos_english.most_common(10))

Unrecognized UD feature 'Compound' with value 'Yes'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized UD feature 'Compound' with value 'Yes'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized UD feature 'Compound' with value 'Yes'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized UD feature 'Compound' with value 'Yes'.
If you believe this is not an error in the dependency parser, please raise an issue at <https://github.com/cltk/cltk/issues> and include a short text to reproduce the error.

Unrecognized UD feature 'Compound' with value 'Yes'.
If you believe this is not an error

In [26]:
!pip uninstall googletrans
!pip install googletrans==4.0.0-rc1

Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.11/dist-packages/googletrans-4.0.0rc1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/googletrans/*
Proceed (Y/n)? y
  Successfully uninstalled googletrans-4.0.0rc1
  Using cached googletrans-4.0.0rc1-py3-none-any.whl


In [28]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [31]:
!pip uninstall -y googletrans
!pip install googletrans==4.0.0-rc1
!pip install deep-translator

Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Successfully uninstalled googletrans-4.0.0rc1
  Using cached googletrans-4.0.0rc1-py3-none-any.whl


In [41]:
# Instalar transformers si no está instalado
!pip install -q transformers
!pip install -q deep-translator

In [42]:
from transformers import pipeline
from deep_translator import GoogleTranslator
from collections import Counter
from random import choice

# Usar todas las palabras (sin top 10)
todas_palabras = list(set(palabras_latin + palabras_english))
todos_verbos = list(set(verbos_latin + verbos_english))

# Función para traducir
def traducir(palabra):
    try:
        return GoogleTranslator(source='auto', target='es').translate(palabra)
    except Exception:
        return palabra  # En caso de error, usar palabra original

# Traducir todas las palabras y verbos
traducciones_palabras = [traducir(p) for p in todas_palabras]
traducciones_verbos = [traducir(v) for v in todos_verbos]


# ✅ Cargar un generador de texto en español
generador = pipeline("text-generation", model="datificate/gpt2-small-spanish")

# ✅ Crear 5 prompts y generar oraciones con IA
oraciones = []
for _ in range(5):
    sust1 = choice(traducciones_palabras)
    sust2 = choice(traducciones_palabras)
    verbo = choice(traducciones_verbos)
    prompt = f"En la vida, cuando uno desea {verbo} la {sust1} y la {sust2},"
    resultado = generador(prompt, max_new_tokens=30, do_sample=True, temperature=0.9)[0]['generated_text']
    oraciones.append(resultado.strip())

# Mostrar resultados
print("\n📜 Oraciones generadas con IA:\n")
for i, o in enumerate(oraciones, 1):
    print(f"{i}. {o}")

Device set to use cpu



📜 Oraciones generadas con IA:

1. En la vida, cuando uno desea oso la verdad y la OMS, hay cuatro objetivos a alcanzar: 



Los objetivos son:

Los métodos de los que se ha aplicado el Método de la Medicina
2. En la vida, cuando uno desea crecer la todo y la por, a la vez se pone a escribir un libro y también se pone a escribir a una chica. 

Al final de la historia, y a partir
3. En la vida, cuando uno desea fio la luz y la dejar, es un acto de servicio. En los siglos XIX y XX, el padre (o "referendor") de la familia fue el médico militar
4. En la vida, cuando uno desea Vincas la Deus y la hombre, que en la mayoría de ocasiones son mujeres, a menudo no se les permite tomar el nombre de "Maestro de Vincas", y por supuesto lo
5. En la vida, cuando uno desea morir la lex y la luz, se da un tiempo de sueño y la luz se pone en una lámpara. Una mujer, en un departamento, que vive cerca de los jardines de la
